# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-10 15:59:08,787 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
! ls

clickstream.csv			   sga-task.ipynb
hdfs-task.ipynb			   sga-task-ndzhassal2.ipynb
LSML_ClickStreams_SGA.ipynb	   sga-task-ndzhassal2-Version2.ipynb
ndzhassal2_spark_sql_solution.csv  spark-warehouse


In [5]:
! hadoop fs -copyFromLocal clickstream.csv /

copyFromLocal: `/clickstream.csv': File exists


In [364]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.functions import concat_ws, collect_list
from pyspark import SparkConf, SparkContext
from pyspark.sql import Window
import pandas as pd
import csv

In [365]:
data = se.read.csv("hdfs:///clickstream.csv", header=True, inferSchema=True, sep = "\t")

In [366]:
type(data)

pyspark.sql.dataframe.DataFrame

In [367]:
# Create a temporary view of the data
data.createOrReplaceTempView("ClickStreams")
data.limit(10).show()

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|   4645|       935|      page|   archive|1696270588|
|   2251|       378|      page|    rabota|1696270591|
|   2222|       704|      page|    rabota|1696270600|
|   2366|       339|      page|     vklad|1696270605|
|   2233|       165|     event|    rabota|1696270605|
|   3253|       528|     event|      news|1696270612|
|    109|       433|     event|     bonus|1696270613|
|   4367|       106|     event|  internet|1696270619|
|   2222|       704|     event|    rabota|1696270628|
|   4304|       640|     event|  internet|1696270636|
+-------+----------+----------+----------+----------+



In [368]:
type(data)

pyspark.sql.dataframe.DataFrame

In [369]:
data.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_page: string (nullable = true)
 |-- timestamp: integer (nullable = true)



In [370]:
data.show(10)
print("Data Types:", data.dtypes)
print("There are {0} rows in the ClickStreams table".format(data.count()))

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
+-------+----------+------------+----------+----------+
only showing top 10 rows

Data Types: [('user_id', 'int'), ('session_id', 'int'), ('event_type', 'string'), ('event_page', 'string'), ('timestamp', 'int')]
There are 1000000 rows in the ClickStreams table


# Part I: Spark SQL (1.0 marks)

- 0.5 points – Spark SQL solution with 1 query
- 0.5 points – Spark SQL solution with <=2 queries

## Task: Use the Spark SQL interface to create a solution file, the lines of which should contain **the 30 most frequent user routes** on the site. 

In [371]:
spark_sql_solution = se.sql("""
SELECT CONCAT_WS('-', route_list) AS route, COUNT(*) AS count
FROM (
    SELECT user_id, session_id, COLLECT_LIST(event_page) AS route_list 
    FROM (
        SELECT cs.user_id, cs.session_id, cs.event_page, cs.timestamp 
        FROM ClickStreams cs
        WHERE cs.event_type = 'page' AND NOT EXISTS (
            SELECT 1 FROM ClickStreams c
            WHERE c.user_id = cs.user_id AND c.session_id = cs.session_id AND c.event_type LIKE '%error%' AND c.timestamp < cs.timestamp
        )
    ) user_routes 
    GROUP BY user_id, session_id
) user_routes
GROUP BY route
ORDER BY count DESC
LIMIT 30;""")

spark_sql_solution.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8090|
|        main-archive| 1088|
|         main-rabota| 1037|
|       main-internet|  879|
|          main-bonus|  865|
|           main-news|  759|
|        main-tariffs|  669|
|         main-online|  584|
|          main-vklad|  512|
| main-archive-rabota|  167|
| main-rabota-archive|  167|
|  main-bonus-archive|  139|
|   main-rabota-bonus|  136|
|    main-news-rabota|  134|
|   main-bonus-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  128|
|   main-archive-news|  124|
|main-rabota-internet|  123|
+--------------------+-----+
only showing top 20 rows



In [372]:
# Save to csv
spark_sql_solution.toPandas().to_csv("ndzhassal2_spark_sql_output.csv", index=False, header=True, sep = "\t") 

In [373]:
Part_1 = pd.read_csv("ndzhassal2_spark_sql_output.csv", sep = "\t")
len(Part_1)

30

# Part II: Spark RDD (0.5 marks)

- 0.5 points – Spark RDD solution

https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

## Task: Use the Spark RDD interface to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.


In [374]:
rdd_data = data.rdd

In [375]:
type(rdd_data)

pyspark.rdd.RDD

In [376]:
# Step 1: Filter 'error' events and map them to tuples in format: ((user_id, session_id), timestamp)
error_tuples = rdd_data.filter(lambda r: 'error' in r.event_type) \
                       .map(lambda r: ((r.user_id, r.session_id), r.timestamp))

# Use reduceByKey() to find the minimum timestamp for each session
errors = error_tuples.reduceByKey(lambda a, b: a if a < b else b)

In [377]:
# Step 2: Filter 'page' events and map them to tuples in format: ((user_id, session_id), event)
page_tuples = rdd_data.filter(lambda r: r.event_type == 'page') \
                      .map(lambda r: ((r.user_id, r.session_id), r))

In [378]:
# Step 3: Join the two RDDs to get tuples in format: ((user_id, session_id), (event, error_timestamp)) 
joined_tuples = page_tuples.leftOuterJoin(errors)

In [379]:
# Step 4: Filter out tuples that contain errors or where event_type = 'page' occurs after an error.
clean_tuples = joined_tuples.filter(lambda x: x[1][1] is None or x[1][0].timestamp < x[1][1])

In [380]:
# Step 5: Mapping clean_tuples to tuples in format: ((user_id, session_id), [event_page])
session_events = clean_tuples.map(lambda x: (x[0], [x[1][0].event_page])) \
                             .reduceByKey(lambda a, b: a + b)

In [381]:
# Step 6: 
# Mapping session_events to add '-' between events 
# Using reduceByKey() to count number of occurrences of each route
user_route_counts = session_events.map(lambda x: ("-".join(x[1]), 1))
user_route_counts = user_route_counts.reduceByKey(lambda a, b: a + b)

In [382]:
# Step 7: # Sort route_counts by count in descending order and take the top 30 results
top_routes = user_route_counts.takeOrdered(30, key=lambda x: -x[1])

In [383]:
type(top_routes)

list

In [384]:
output_file = "ndzhassal2_spark_RDD_output.csv"
cols = ["route", "count"]

In [385]:
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(cols)
    writer.writerows(top_routes)

# Part III: Spark DF (0.5 marks)

- 0.5 points – Spark DF solution

## Task: Use the Spark DF interface to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

In [386]:
print("Before conversion:", type(rdd_data))

Before conversion: <class 'pyspark.rdd.RDD'>


In [387]:
# Coverting rdd_data to Spark DataFrame
clickstream_df = rdd_data.toDF()

In [388]:
print("After conversion:", type(clickstream_df))

After conversion: <class 'pyspark.sql.dataframe.DataFrame'>


In [389]:
clickstream_df.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



In [390]:
clickstream_df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- session_id: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_page: string (nullable = true)
 |-- timestamp: long (nullable = true)



In [391]:
# Step 1: Filter errors and calculate min_timestamp
errors_df = (
    clickstream_df
    .filter(F.col('event_type').like('%error%'))  
    .groupBy('user_id', 'session_id')  
    .agg(F.min('timestamp').alias('min_timestamp'))  # Calculate minimum value of timestamp for each group
)

In [392]:
# Step 2: Join tables and filter data
user_sessions_df = (
    clickstream_df
    .join(errors_df, ['user_id', 'session_id'], how='left')  # Join with errors_df on user_id and session_id columns
    .filter(((F.col('timestamp') < F.col('min_timestamp')) | F.col('min_timestamp').isNull()) & (F.col('event_type') == 'page'))  # Keep rows where timestamp is less than min_timestamp or min_timestamp is NULL, and event_type is "page"
    .withColumn('rn', F.row_number().over(Window.partitionBy('user_id', 'session_id').orderBy(F.col('timestamp').desc())))  # Add a new column "rn" that assigns a row number to each row within each group, based on the descending order of the timestamp column
)

In [393]:
# Step 3: Collect user routes and filter data
filtered_user_routes_df = (
    user_sessions_df
    .groupBy('user_id', 'session_id')
    .agg(F.collect_list('event_page').alias('route'), F.first('timestamp').alias('max_timestamp'))  
    .withColumn('rn', F.row_number().over(Window.partitionBy('user_id', 'session_id').orderBy(F.col('max_timestamp').desc())))  
    .filter(F.col('rn') == 1)
)

In [394]:
# Step 4: Group by routes and count
all_routes = (
    filtered_user_routes_df
    .groupBy('route')
    .agg(F.count('*').alias('count'))
    .orderBy('count', ascending=False)  # Sort in descending order
)

all_routes.show(30) 

+--------------------+-----+
|               route|count|
+--------------------+-----+
|              [main]| 8090|
|     [main, archive]| 1094|
|      [main, rabota]| 1037|
|    [main, internet]|  880|
|       [main, bonus]|  865|
|        [main, news]|  760|
|     [main, tariffs]|  669|
|      [main, online]|  584|
|       [main, vklad]|  514|
|[main, archive, r...|  167|
|[main, rabota, ar...|  167|
|[main, bonus, arc...|  139|
|[main, rabota, bo...|  137|
|[main, news, rabota]|  134|
|[main, bonus, rab...|  133|
|[main, archive, i...|  131|
|[main, rabota, news]|  129|
|[main, internet, ...|  128|
|[main, archive, n...|  124|
|[main, internet, ...|  123|
|[main, rabota, in...|  123|
|[main, archive, b...|  117|
|[main, internet, ...|  114|
|[main, tariffs, i...|  114|
|[main, news, arch...|  112|
|[main, news, inte...|  108|
|[main, archive, t...|  103|
|[main, tariffs, a...|  102|
|[main, internet, ...|  101|
|        [main, main]|   94|
+--------------------+-----+
only showing t

In [395]:
type(all_routes)

pyspark.sql.dataframe.DataFrame

In [396]:
output_file = "ndzhassal2_spark_DF_output.csv"
cols = ["route", "count"]

In [397]:
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(cols)
    writer.writerows(all_routes.collect()) 